#Хеш-функции и хеш-таблицы.
Для начала разберёмся с тем, что такое хеш-функция.
Если очень просто и коротко, то это функция, которая очень сильно "перемешивает" исходные данные и создаён некую новую строку. Так, что 1) исходные данные востановить фактически невозможно, кроме как брутфорсом, 2) если знать исходные данные, то сама хеш-функция всегда вычисляется одинаково.

Самих функций есть великое множество, сегодня мы не будем говорить об их преймуществах и непосредственно методах работы.

###Задача 1.
Путешественник во времени (с хорошей памятью) хочет доказать, что он правда из будущего. Но он не имеет права сообщать никакую сколь-нибудь полезную информацию о будущем. Как ему поступить?


###Задача 2. (хочется, чтобы её решение стало очевидно для всех)
Программист разрабатывает базу данных, в которой будут храниться данные аутентификации пользователей некоторого сервиса. Пользователи вводят логин и пароль, а система должна понять, являются ли они корректной парой. Но программист (вполне обоснованно) не очень доверяет Администратору и боится, что базу украдут. Как Программисту сделать так, чтобы пароли пользователей никто не узнал, даже если базу украдут?

### Задача 3*.
Злоумышленник украл нашу базу, но не смог сразу узнать пароли (задача 2). Но он не отчаивается, ведь известно, что около 36% людей используют словарные пароли (погуглите, что это такое). А база словарных паролей у злоумышленника, конечно, есть. Как Программист мог бы максимально усложнить задачу Злоумышленника?

###Как вызвать хеш-функцию в питоне?
Например, следующим образом...

In [ ]:
from hashlib import sha256

s = "Hello World"
s_b = bytearray(s, "UTF-8")
sha_res = sha256(s_b).hexdigest() # у всех должно получаться одно и то же

In [ ]:
int_hash = int(sha_res, 16)
int_hash

74888964247292943290829644364954473609342749251111522634754282069725240300654

Или намнооооого более простым способом.

In [ ]:
hash(5)

5

Почему получились разные значения? Кстати, а ко всему ли можно применять функцию hash()?

###Так что такое хеш-таблица? (а заодно, как устроен set в python)
Обычно это такой массив `hash_table`, в котором информация об элементе `x` хранится в `hash_table[hash_function(x) % len(hash_table)]`.

Несложно понять, что таким образом очень удобно хранить информацию о неком множестве. За одну операцию мы можем проверять, есть ли элемент в таблице, удалять его или добавлять.

В чём состоят фундаментальные проблемы такого подхода?

Я не буду спойлерить прям в тексте ответ на предыдущий вопрос. Но если захотите почитать, то основная проблема - коллизии и их можно разрешать цепочками.

###Задача 4.
Напишите свой класс `hash_table` в котором будут рализованы методы (к хранению допускаются только данные immutable-типов. Если вы не помните, что это такое, давайте обсудим).

* `.__init__()`
* `.add()`
* `.remove()`


In [ ]:
%%file map.py

LEN = 100


class hashIterator():
    def __init__(self, data):
        self.ind = 0
        self.subInd = 0
        self.data = data

    def __iter__(self):
        return self

    def __next__(self):
        if self.ind + 1 >= len(self.data):
            raise StopIteration

        if self.subInd >= len(self.data[self.ind]):
            self.ind += 1
            self.subInd = 0

        if self.data[self.ind] != []:
            self.subInd += 1
            return self.data[self.ind][self.subInd - 1]
        else:
            return self.__next__()


class hashTable():
    def __init__(self):
        self.data = [[] for _ in range(LEN)]
        self.length = 0

    def add(self, elem):
        for num in self.data[hash(elem) % LEN]:
            if (num == elem):
                return

        self.length += 1
        self.data[hash(elem) % LEN].append(elem)

    def remove(self, elem):
        arr = self.data[hash(elem) % LEN]
        for i in range(len(arr)):
            if (arr[i] == elem):
                arr.pop(i)
                self.length -= 1
                return

    def __len__(self):
        return self.length

    def __contains__(self, elem):
        return elem in self.data[hash(elem) % LEN]

    def __iter__(self):
        return hashIterator(self.data)


Writing map.py


In [ ]:
# Ignore this block of code
from map import *

table = hashTable()
table.add('hello')
table.add('card')
table.add('there')

for elem in table:
    print(elem)

hello
there
card


###Задача 5.
Сделайте так, чтобы с вашим классом корректно работали функция **len** и ключевое слово **in**.

###Задача 6*.
Сделайте так, чтобы для элементов вашей таблицы корректно работал цикл **for**.

**UPD.** Хорошей идеей будет вынести ```__iter__``` в отдельный класс.

###Задача 7*.
Сделайте предварительную проверку на наличие элемента с помощью [фильтра Блума](https://neerc.ifmo.ru/wiki/index.php?title=%D0%A4%D0%B8%D0%BB%D1%8C%D1%82%D1%80_%D0%91%D0%BB%D1%83%D0%BC%D0%B0).

Хотя, кажется, это не совсем та ситуация, когда фильтр полезен. Он скорее хорош, когда таблица реально большая, в ней много записей и часто приходится искать не существующие значения. Можете на любом языке написать фильтр и сымитировать нагрузку так, чтобы фильтр работал лучше стандартного контейнера.

#Финальный лоск.

###Задача 8.

Убедитесь, что на все поля и методы в вашем классе выставлена корректная приватность-публичность.

###Задача 9.
Напишите скрипт по тестированию для pytest. (В спорных ситуациях ваша структура должна работать как стандартный set)

In [ ]:
!pip -q install pytest pytest-sugar Random-Word-Generator

# move to tdd directory
from pathlib import Path
if Path.cwd().name != 'tdd':
    %mkdir tdd
    %cd tdd

%pwd

'/content/tdd'

In [ ]:
# PYTEST
%%file test.py

from map import *
import pytest
from RandomWordGenerator import RandomWord


def test_add():
    try:
        table = hashTable()
        table.add('hello')
    except Exception as Error:
        pytest.fail(Error)


def test_remove():
    try:
        table = hashTable()
        table.add('hello')
        table.remove('hello')
        if table.data != [[] for _ in range(LEN)]:
            pytest.fail('There are elements in hash table')
    except Exception as Error:
        pytest.fail(Error)


def test_for():
    try:
        table = hashTable()
        table.add('hello')
        table.add('there')
        for elem in table:
            if elem != 'hello' and elem != 'there':
                pytest.fail('Element in for loop isnt appearing in hashtable')
    except Exception as Error:
        pytest.fail(Error)


def test_len():
    try:
        table = hashTable()
        table.add('hello')
        table.add('there')
        if len(table) != 2:
            pytest.fail('Length is calculated wrong')
    except Exception as Error:
        pytest.fail(Error)


def test_in():
    try:
        table = hashTable()
        table.add('hello')
        if 'hello' not in table:
            pytest.fail('Contains doesnt work propperly')
    except Exception as Error:
        pytest.fail(Error)


def test_random_for():
    try:
        arr = []
        table = hashTable()

        for i in range(10):
            arr.append(RandomWord())
            table.add(arr[i])
        for elem in table:
            if elem == [] or elem not in arr:
                pytest.fail('Element in for loop isnt appearing in hashtable')

        if len(table) != 10:
            pytest.fail('Length is calculated wrong')

    except Exception as Error:
        pytest.fail(Error)


Overwriting test.py


In [ ]:
!python -m pytest test.py 

Test session starts (platform: linux, Python 3.6.9, pytest 3.6.4, pytest-sugar 0.9.4)
rootdir: /content/tdd, inifile:
plugins: typeguard-2.7.1, sugar-0.9.4

 test.py ✓✓✓✓✓✓                                                  100% ██████████

Results (0.04s):
       6 passed


###Задача 10.
Получите 10/10 по PEP8 и залейте код на гитхаб.

In [ ]:
# Experiment zone

words = '''
ability
able
about
above
accept
according
account
across
act
action
activity
actually
add
address
administration
admit
adult
affect
after
again
against
age
agency
agent
ago
agree
agreement
ahead
air
all
allow
almost
alone
along
already
also
although
always
American
among
amount
analysis
and
animal
another
answer
any
anyone
anything
appear
apply
approach
area
argue
arm
around
arrive
art
article
artist
as
ask
assume
at
attack
attention
attorney
audience
author
authority
available
avoid
away
baby
back
bad
bag
ball
bank
bar
base
be
beat
beautiful
because
become
bed
before
begin
behavior
behind
believe
benefit
best
better
between
beyond
big
bill
billion
bit
black
blood
blue
board
body
book
born
both
box
boy
break
bring
brother
budget
build
building
business
but
buy
by
call
camera
campaign
can
cancer
candidate
capital
car
card
care
career
carry
case
catch
cause
cell
center
central
century
certain
certainly
chair
challenge
chance
change
character
charge
check
child
choice
choose
church
citizen
city
civil
claim
class
clear
clearly
close
coach
cold
collection
college
color
come
commercial
common
community
company
compare
computer
concern
condition
conference
Congress
consider
consumer
contain
continue
control
cost
could
country
couple
course
court
cover
create
crime
cultural
culture
cup
current
customer
cut
dark
data
daughter
day
dead
deal
death
debate
decade
decide
decision
deep
defense
degree
Democrat
democratic
describe
design
despite
detail
determine
develop
development
die
difference
different
difficult
dinner
direction
director
discover
discuss
discussion
disease
do
doctor
dog
door
down
draw
dream
drive
drop
drug
during
each
early
east
easy
eat
economic
economy
edge
education
effect
effort
eight
either
election
else
employee
end
energy
enjoy
enough
enter
entire
environment
environmental
especially
establish
even
evening
event
ever
every
everybody
everyone
everything
evidence
exactly
example
executive
exist
expect
experience
expert
explain
eye
face
fact
factor
fail
fall
family
far
fast
father
fear
federal
feel
feeling
few
field
fight
figure
fill
film
final
finally
financial
find
fine
finger
finish
fire
firm
first
fish
five
floor
fly
focus
follow
food
foot
for
force
foreign
forget
form
former
forward
four
free
friend
from
front
full
fund
future
game
garden
gas
general
generation
get
girl
give
glass
go
goal
good
government
great
green
ground
group
grow
growth
guess
gun
guy
hair
half
hand
hang
happen
happy
hard
have
he
head
health
hear
heart
heat
heavy
help
her
here
herself
high
him
himself
his
history
hit
hold
home
hope
hospital
hot
hotel
hour
house
how
however
huge
human
hundred
husband
I
idea
identify
if
image
imagine
impact
important
improve
in
include
including
increase
indeed
indicate
individual
industry
information
inside
instead
institution
interest
interesting
international
interview
into
investment
involve
issue
it
item
its
itself
job
join
just
keep
key
kid
kill
kind
kitchen
know
knowledge
land
language
large
last
late
later
laugh
law
lawyer
lay
lead
leader
learn
least
leave
left
leg
legal
less
let
letter
level
lie
life
light
like
likely
line
list
listen
little
live
local
long
look
lose
loss
lot
love
low
machine
magazine
main
maintain
major
majority
make
man
manage
management
manager
many
market
marriage
material
matter
may
maybe
me
mean
measure
media
medical
meet
meeting
member
memory
mention
message
method
middle
might
military
million
mind
minute
miss
mission
model
modern
moment
money
month
more
morning
most
mother
mouth
move
movement
movie
Mr
Mrs
much
music
must
my
myself
name
nation
national
natural
nature
near
nearly
necessary
need
network
never
new
news
newspaper
next
nice
night
no
none
nor
north
not
note
nothing
notice
now
n't
number
occur
of
off
offer
office
officer
official
often
oh
oil
ok
old
on
once
one
only
onto
open
operation
opportunity
option
or
order
organization
other
others
our
out
outside
over
own
owner
page
pain
painting
paper
parent
part
participant
particular
particularly
partner
party
pass
past
patient
pattern
pay
peace
people
per
perform
performance
perhaps
period
person
personal
phone
physical
pick
picture
piece
place
plan
plant
play
player
PM
point
police
policy
political
politics
poor
popular
population
position
positive
possible
power
practice
prepare
present
president
pressure
pretty
prevent
price
private
probably
problem
process
produce
product
production
professional
professor
program
project
property
protect
prove
provide
public
pull
purpose
push
put
quality
question
quickly
quite
race
radio
raise
range
rate
rather
reach
read
ready
real
reality
realize
really
reason
receive
recent
recently
recognize
record
red
reduce
reflect
region
relate
relationship
religious
remain
remember
remove
report
represent
Republican
require
research
resource
respond
response
responsibility
rest
result
return
reveal
rich
right
rise
risk
road
rock
role
room
rule
run
safe
same
save
say
scene
school
science
scientist
score
sea
season
seat
second
section
security
see
seek
seem
sell
send
senior
sense
series
serious
serve
service
set
seven
several
sex
sexual
shake
share
she
shoot
short
shot
should
shoulder
show
side
sign
significant
similar
simple
simply
since
sing
single
sister
sit
site
situation
six
size
skill
skin
small
smile
so
social
society
soldier
some
somebody
someone
something
sometimes
son
song
soon
sort
sound
source
south
southern
space
speak
special
specific
speech
spend
sport
spring
staff
stage
stand
standard
star
start
state
statement
station
stay
step
still
stock
stop
store
story
strategy
street
strong
structure
student
study
stuff
style
subject
success
successful
such
suddenly
suffer
suggest
summer
support
sure
surface
system
table
take
talk
task
tax
teach
teacher
team
technology
television
tell
ten
tend
term
test
than
thank
that
the
their
them
themselves
then
theory
there
these
they
thing
think
third
this
those
though
thought
thousand
threat
three
through
throughout
throw
thus
time
to
today
together
tonight
too
top
total
tough
toward
town
trade
traditional
training
travel
treat
treatment
tree
trial
trip
trouble
true
truth
try
turn
TV
two
type
under
understand
unit
until
up
upon
us
use
usually
value
various
very
victim
view
violence
visit
voice
vote
wait
walk
wall
want
war
watch
water
way
we
weapon
wear
week
weight
well
west
western
what
whatever
when
where
whether
which
while
white
who
whole
whom
whose
why
wide
wife
will
win
wind
window
wish
with
within
without
woman
wonder
word
work
worker
world
worry
would
write
writer
wrong
yard
yeah
year
yes
yet
you
young
your
yourself
'''

In [ ]:
hash('hello') % 100

for w in words.split():
    if hash(w) % 100 == hash('hello') % 100:
        print(w)